<a href="https://colab.research.google.com/github/leksandre/-crab-for-web2ipr_book_shop/blob/main/mocsTrainYOLOv3ForYoCol(normal_work).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training YOLOv3 From Scratch with Darknet

In [ ]:
from google.colab import drive
drive.mount('/content/')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


AttributeError: module 'google.colab.drive' has no attribute 'unmount'

In [ ]:
! cd /content/drive/MyDrive/

In [ ]:
! cd /content/drive/MyDrive/mocs_dataset/

In [ ]:
%ls ./

drive/  sample_data/


In [ ]:
!pwd

/content


In [ ]:
!git clone https://github.com/Oskop/YoCol

Cloning into 'YoCol'...
remote: Enumerating objects: 1172, done.
remote: Total 1172 (delta 0), reused 0 (delta 0), pack-reused 1172
Receiving objects: 100% (1172/1172), 11.32 MiB | 26.96 MiB/s, done.
Resolving deltas: 100% (281/281), done.


## Setup Environment For Darknet

Since Google Colab has pre-installed CUDA 10, then we can skip CUDA configuration and go to next step

In [ ]:
#Installing compilers
!apt install gcc-5 g++-5 -y

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Package g++-5 is not available, but is referred to by another package.
This may mean that the package is missing, has been obsoleted, or
is only available from another source
However the following packages replace it:
  gcc-10-test-results gcc-9-test-results gcc-12-test-results gcc-11-test-results

E: Unable to locate package gcc-5
E: Package 'g++-5' has no installation candidate


In [ ]:
!ln -s /usr/bin/gcc-5 /usr/local/cuda/bin/gcc
!ln -s /usr/bin/g++-5 /usr/local/cuda/bin/g++

In [ ]:
#Changing the variables to include OpenCV and GPU in the Makefile
! cd /content/YoCol/darknet/
!sed -i 's/OPENCV=0/OPENCV=1/g' Makefile
!sed -i 's/GPU=0/GPU=1/g' Makefile
#!sed -i 's/CUDNN=0/CUDNN=1/g' Makefile if you want to use cudnn

sed: can't read Makefile: No such file or directory
sed: can't read Makefile: No such file or directory


In [ ]:
# Apparently we need to install this so that OpenCV can work without any issues
# when we are making the file
!apt-get install libopencv-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libopencv-dev is already the newest version (4.5.4+dfsg-9ubuntu4+jammy0).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


## Get Dataset

### Download and Extract Challenge Dataset

Downloading and extracting Standford car dataset and extracting to /content

In [ ]:
%cd /content
!wget http://imagenet.stanford.edu/internal/car196/cars_train.tgz
!wget http://imagenet.stanford.edu/internal/car196/cars_test.tgz
!wget https://ai.stanford.edu/~jkrause/cars/car_devkit.tgz
!tar zxf car_devkit.tgz
!tar zxf cars_train.tgz
!tar zxf cars_test.tgz
!wget http://imagenet.stanford.edu/internal/car196/cars_test_annos_withlabels.mat -O devkit/cars_test_annos_withlabels.mat

/content
--2024-03-01 10:40:15--  http://imagenet.stanford.edu/internal/car196/cars_train.tgz
Resolving imagenet.stanford.edu (imagenet.stanford.edu)... 171.64.68.16
Connecting to imagenet.stanford.edu (imagenet.stanford.edu)|171.64.68.16|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://image-net.org/internal/car196/cars_train.tgz [following]
--2024-03-01 10:40:15--  https://image-net.org/internal/car196/cars_train.tgz
Resolving image-net.org (image-net.org)... 171.64.68.16
Connecting to image-net.org (image-net.org)|171.64.68.16|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2024-03-01 10:40:16 ERROR 404: Not Found.

--2024-03-01 10:40:16--  http://imagenet.stanford.edu/internal/car196/cars_test.tgz
Resolving imagenet.stanford.edu (imagenet.stanford.edu)... 171.64.68.16
Connecting to imagenet.stanford.edu (imagenet.stanford.edu)|171.64.68.16|:80... connected.
HTTP request sent, awaiting response... 301 Moved Pe

###Download and Extract COCO Dataset

Include COCO dataset that handled with get_coco_dataset.sh script so we don't need to convert label format from COCO format to YOLOv3 format. This step is an optional so you can skip if you think there's no need to including COCO dataset into training process. But if you not include COCO dataset, then you must use yolov3.weights from official site to continue the training process because the number of classes for this config is including COCO 80 classes

In [ ]:
%cd /content/YoCol/darknet
!sh scripts/get_coco_dataset.sh

[Errno 2] No such file or directory: '/content/YoCol/darknet'
/content
sh: 0: cannot open scripts/get_coco_dataset.sh: No such file


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/YoCol/darknet/coco
!paste <(awk "{pri!nt \"$PWD\"}" <5k.part) 5k.part | tr -d '\t' > 5k.txt
!paste <(awk "{print \"$PWD\"}" <trainvalno5k.part) trainvalno5k.part | tr -d '\t' > trainvalno5k.txt
%cd /content

[Errno 2] No such file or directory: '/content/YoCol/darknet/coco'
/content
paste: 5k.part: No such file or directory
/bin/bash: line 1: 5k.part: No such file or directory
paste: trainvalno5k.part: No such file or directory
/bin/bash: line 1: trainvalno5k.part: No such file or directory
/content


### Labeling

We load the MATLAB annotations file first, then adding to a new dictionary that store both the Standford labels and COCO labels.

In [ ]:
%cd /content
import scipy.io as scio
import os
import cv2
matlab_train_label = scio.loadmat('devkit/cars_train_annos.mat')
matlab_test_label = scio.loadmat('devkit/cars_test_annos_withlabels.mat')
YOLO_CLASS = 79
TRAIN_IMAGE_PATH = '/content/cars_train/'
TEST_IMAGE_PATH = '/content/cars_test/'

/content


FileNotFoundError: [Errno 2] No such file or directory: 'devkit/cars_train_annos.mat'

In [ ]:
# Store train labels data to another list for easy calling data
file_names_train = matlab_train_label['annotations']['fname'][0]
obj_classes_train = matlab_train_label['annotations']['class'][0]
bboxes_x1_train = matlab_train_label['annotations']['bbox_x1'][0]
bboxes_x2_train = matlab_train_label['annotations']['bbox_x2'][0]
bboxes_y1_train = matlab_train_label['annotations']['bbox_y1'][0]
bboxes_y2_train = matlab_train_label['annotations']['bbox_y2'][0]

In [ ]:
# Store test labels data to another list for easy calling data
file_names_test = matlab_test_label['annotations']['fname'][0]
obj_classes_test = matlab_test_label['annotations']['class'][0]
bboxes_x1_test = matlab_test_label['annotations']['bbox_x1'][0]
bboxes_x2_test = matlab_test_label['annotations']['bbox_x2'][0]
bboxes_y1_test = matlab_test_label['annotations']['bbox_y1'][0]
bboxes_y2_test = matlab_test_label['annotations']['bbox_y2'][0]

In [ ]:
# load labels that i've already annotate to train and test
# standford images with YOLOv3 (contain 80 classes of COCO classes,
# 196 classes from Sandford classes are not include in this data.json file)
import json
cars_make_model_label = open('/content/YoCol/data.json', 'r')
standford_label = json.load(cars_make_model_label)

In [ ]:
# Initialize dictionary that will contain all label from both labels data
for names in standford_label:
  all_label[names] = []

In [ ]:
# Changing test label file name from "file_names_test" list to adjust
# the key format from data.json
for name in file_names_test:
  for file_name in name:
    if (int(file_name.split('.')[0])+8144) < 10000:
      new_name = '0' + str(int(file_name.split('.')[0]) + 8144) + '.jpg'
      name[0] = new_name
      os.rename(TEST_IMAGE_PATH + file_name, TEST_IMAGE_PATH + new_name)
    else:
      new_name = str(int(file_name.split('.')[0]) + 8144) + '.jpg'
      name[0] = new_name
      os.rename(TEST_IMAGE_PATH + file_name, TEST_IMAGE_PATH + new_name)

In [ ]:
# Looping into train labels data and change its format to YOLO annotation format
for (i, name) in enumerate(file_names_train):
  file_name = name[0]
  image = cv2.imread(TRAIN_IMAGE_PATH + file_name)
  height = image.shape[0]
  width = image.shape[1]
  if image.shape[1] >= 300:
      obj_class = str(obj_classes[i][0][0] + YOLO_CLASS)
      x1 = float(bboxes_x1_train[i][0])
      x2 = float(bboxes_x2_train[i][0])
      y1 = float(bboxes_y1_train[i][0])
      y2 = float(bboxes_y2_train[i][0])
      x_center = str(((x2-x1) / 2) / width)
      y_center = str(((y2-y1) / 2) / height)
      obj_width = str((x2-x1) / width)
      obj_height = str((y2-y1) / height)
      all_label[file_name].append(obj_class + ' ' + x_center + ' ' + y_center +
                                  ' ' + obj_width + ' ' + obj_height)

In [ ]:
# Looping into test labels data and change its format to YOLO annotation format
for (i, name) in enumerate(file_names_test):
  file_name2 = name[0]
  image = cv2.imread(TEST_IMAGE_PATH + file_name2)
  height = image.shape[0]
  width = image.shape[1]
  if image.shape[1] >= 300:
      obj_class = str(obj_classes[i][0][0] + YOLO_CLASS)
      x1 = float(bboxes_x1_test[i][0])
      x2 = float(bboxes_x2_test[i][0])
      y1 = float(bboxes_y1_test[i][0])
      y2 = float(bboxes_y2_test[i][0])
      x_center = str(((x2-x1) / 2) / width)
      y_center = str(((y2-y1) / 2) / height)
      obj_width = str((x2-x1) / width)
      obj_height = str((y2-y1) / height)
      all_label[file_name2].append(obj_class + ' ' + x_center + ' ' + y_center +
                                  ' ' + obj_width + ' ' + obj_height)

In [ ]:
# Loop over standford_label and adjust annotation format to YOLO format
import cv2
for file_name in standford_label:
#   if file_name in train_file_directory:
  for bbox in standford_label[file_name]:
    image = cv2.imread(TRAIN_IMAGE_PATH + file_name)
    height = image.shape[0]
    width = image.shape[1]
    if bbox[0] < 0:
      left = 0
    else:
      left = bbox[0]
    if bbox[1] < 0:
      top = 0
    else:
      top = bbox[1]
    if bbox[2] < 0:
      right = 1
    else:
      right = bbox[2]
    if bbox[3] < 0:
      bottom = 1
    else:
      bottom = bbox[3]
    x_center = str(((right - left) / 2) / width)
    y_center = str(((bottom - top) / 2) / height)
    obj_width = str((right - left) / width)
    obj_height = str((bottom - top) / height)
    obj_class = str(bbox[4])
    all_label[file_name].append(obj_class + ' ' + x_center + ' ' + y_center
                                + ' ' + obj_width + ' ' + obj_height)

Then we move COCO Training and Validation images and its labels into Standford Images Directory. Labels data are moved to image directory because darknet will detect our label in same directory as images

In [ ]:
# Moving COCO Images into Standford Images Directory
!mv /content/YoCol/darknet/coco/images/train2014/* /content/cars_train
!mv /content/YoCol/darknet/coco/images/val2014/* /content/casr_test

In [ ]:
# # Moving COCO Labels into Standford Images Directory
!mv /content/YoCol/darknet/coco/labels/train2014/* /content/cars_train
!mv /content/YoCol/darknet/coco/labels/val2014/* /content/cars_test

Darknet need to a list file of image file path that will be passes to darknet for training and validation

In [ ]:
# Make list file of train and test image file path
train_images_list = os.listdir(TRAIN_IMAGE_PATH)
test_images_list = os.listdir(TEST_IMAGE_PATH)
train_list_file = open('/content/train_list.txt', 'w')
test_list_file = open('/content/test_list.txt', 'w')
for img_name in train_images_list:
  train_list_file.write(TRAIN_IMAGE_PATH + img_name + '\n')
for img_name2 in test_images_list:
  test_list_file.write(TEST_IMAGE_PATH + img_name2 + '\n')

In [ ]:
# Then we start labelling Standford Dataset based on labels data that stored in
# "all_label" dictionary
for name in all_label:
  if int(name[:-4]) < 8145 :
    label_file = open(TRAIN_IMAGE_PATH + name[:-4] + '.txt', 'w')
  else:
    label_file = open(TEST_IMAGE_PATH + name[:-4] + '.txt', 'w')
  for label in all_label[name]:
    label_file.write(label + '\n')

##Setting before Training for Yolov3

Darknet need some configuration file befor training YOLO model that had ".data" extension. This file contains some configuration such as where darknet must take list file of training and validation, classes names that will use for YOLO, and path to store .weights file

In [ ]:
!mkdir /content/weight
dat = """classes = 196
train = /content/train_list.txt
valid = /content/test_list.txt
names = /content/YoCol/data/yocol.names
backup = /content/"""
with open('/content/darknet.data','w') as dark:
  dark.write(dat)

####Compile Darknet

In [ ]:
%cd /content/YoCol/darknet
!make
#Check if darknet is installed properly
!./darknet detector help

##Start Training Yolov3

The training is start from scratch because we include both dataset and want to not only detect Standford Car classes name but also 80 COCO classes name

In [ ]:
%cd /content/YoCol/darknet/
!./darknet detector train /content/darknet.data cfg/yolov3.cfg &> /dev/null